In [16]:

!echo '数据集行数:'
!wc -l 'data/bioCorpus_5000.txt'
!echo '======'
!echo '数据集前10行'
!head -10 'data/bioCorpus_5000.txt'


数据集行数:
5000 data/bioCorpus_5000.txt
数据集前10行
formate assay in body fluids  application in methanol poisoning.
delineation of the intimate details of the backbone conformation of pyridinenucleotide coenzymes in aqueous solution.
metal substitutions incarbonic anhydrase  a halide ion probe study.
effect of chloroquine on cultured fibroblasts  release of lysosomal hydrolasesand inhibition of their uptake.
atomic models for the polypeptide backbones of myohemerythrin and hemerythrin.
studies of oxygen binding energy to hemoglobin molecule.
maturation of the adrenal medulla--IV
effects of morphine.
comparison between procaine and isocarboxazid metabolism in vitro by a livermicrosomal amidase-esterase.
radiochemical assay of glutathione S-epoxide transferase and its enhancement byphenobarbital in rat liver in vivo.


In [17]:
from gensim.models import word2vec 

In [18]:
# 用生成器的方式读取文件里的句子# 适合读取大容量文件，而不用加载到内存
class MySentences(object):
    def __init__(self, fname):
        self.fname = fname    
    def __iter__(self):
        for line in open(self.fname,'r'):     
            yield line.split()

In [19]:
# 模型训练函数
def w2vTrain(f_input, model_output):         
    sentences = MySentences(DataDir+f_input)
    w2v_model = word2vec.Word2Vec(sentences, 
                                  min_count = MIN_COUNT, 
                                  workers = CPU_NUM, 
                                  size = VEC_SIZE,
                                  window = CONTEXT_WINDOW
                                 )

    w2v_model.save(ModelDir+model_output)

In [23]:
# 训练
DataDir = "./data/"
ModelDir = "./ipynb_garbage_files/"
MIN_COUNT = 4
CPU_NUM = 2 
# 需要预先安装 Cython 以支持并行
VEC_SIZE = 20
CONTEXT_WINDOW = 5 # 提取目标词上下文距离最长5个词
f_input = "bioCorpus_5000.txt"
model_output = "test_w2v_model"

w2vTrain(f_input, model_output)

In [24]:
w2v_model = word2vec.Word2Vec.load(ModelDir+model_output)

In [25]:
w2v_model.most_similar('body')

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('blocking', 0.9992074966430664),
 ('human', 0.9991682767868042),
 ('response', 0.9990294575691223),
 ('inhibitor', 0.999004602432251),
 ('and', 0.9989744424819946),
 ('vitro', 0.9989644885063171),
 ('activity', 0.998931884765625),
 ('from', 0.9989229440689087),
 ('during', 0.9989050626754761),
 ('system', 0.998888373374939)]

In [26]:
w2v_model.most_similar('heart') 


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('at', 0.9995989203453064),
 ('assay', 0.9995205402374268),
 ('The', 0.9994934797286987),
 ('a', 0.9994790554046631),
 ('cytochrome', 0.9994041919708252),
 ('cells', 0.9993923306465149),
 ('acid', 0.9993584752082825),
 ('letter', 0.9993261694908142),
 ('plasma', 0.9993071556091309),
 ('an', 0.9992932081222534)]

In [30]:
# 停止词
from nltk.corpus import stopwords
StopWords = stopwords.words('english')

In [31]:
StopWords[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [32]:
# 重新训练# 模型训练函数
def w2vTrain_removeStopWords(f_input, model_output):         
    sentences = list(MySentences(DataDir+f_input))    
    for idx,sentence in enumerate(sentences):

        sentence = [w for w in sentence if w not in StopWords]

        sentences[idx]=sentence

    w2v_model = word2vec.Word2Vec(sentences, 
                                  min_count = MIN_COUNT, 
                                  workers = CPU_NUM, 
                                  size = VEC_SIZE)

    w2v_model.save(ModelDir+model_output)

w2vTrain_removeStopWords(f_input, model_output)

w2v_model = word2vec.Word2Vec.load(ModelDir+model_output)

In [36]:
w2v_model.most_similar('body')

# 结果一般
# [('relationship', 0.9543654918670654),
#   ('plasma', 0.9490970373153687),

#  ('two', 0.9482829570770264),

#  ('blood', 0.9451138973236084),

#  ('structure', 0.9415417909622192),

#  ('properties', 0.9410394430160522),

#  ('human', 0.9409817457199097),

#  ('cardiac', 0.9402023553848267),

#  ('effect', 0.9401187896728516),

#  ('response', 0.9397702217102051)]

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('blocking', 0.9652001261711121),
 ('inhibitor', 0.9612349271774292),
 ('human', 0.9593227505683899),
 ('response', 0.9581257104873657),
 ('vitro', 0.9542000889778137),
 ('different', 0.952316403388977),
 ('function', 0.9517786502838135),
 ('system', 0.950149416923523),
 ('action', 0.9496978521347046),
 ('renal', 0.9496498107910156)]

In [37]:
w2v_model.most_similar('heart')

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('The', 0.9849681258201599),
 ('assay', 0.9808926582336426),
 ('cells', 0.9777134656906128),
 ('blood', 0.9770573377609253),
 ('beta', 0.9767102599143982),
 ('acid', 0.9757313132286072),
 ('determination', 0.9754316806793213),
 ('response', 0.9754077196121216),
 ('changes', 0.9747826457023621),
 ('plasma', 0.9747099876403809)]